In [ ]:
PRICE = [[100, 70, 40, 0], [70, 50, 30, 0], [40, 30, 20, 0], [0, 0, 0, 0]]
def go(G, N, M, DP, i, B) :
  if i >= N * M :
    return 0

  if DP[i][B] != -1 :
    return DP[i][B]

  cmp = go(G, N, M, DP, i + 1, B >> 1)
  DP[i][B] = max(DP[i][B], cmp)

  if B & 1 == 0 :
    if i + M < N * M: 
      cmp = go(G, N, M, DP, i + 1, B >> 1 | (1 << (M-1))) 
      DP[i][B] = max(DP[i][B], cmp + PRICE[G[i // M][i % M]][G[(i + M) // M][(i + M) % M]])
    
    if i % M != (M-1) and B & 2 == 0 :
      cmp = go(G, N, M, DP, i + 2, B >> 2)
      DP[i][B] = max(DP[i][B], cmp + PRICE[G[i // M][i % M]][G[(i + 1) // M][(i + 1) % M]])
  return DP[i][B]

def sol() :
  N = int(input())
  G = [[*map(lambda x: ord(x) - 65 if x != 'F' else 3, input().rstrip())] for _ in range(N)]
  DP = [[-1] * (1 << N) for _ in range(N * N)]
  go(G, N, N, DP, 0, 0)
  print(DP[0][0])
  
sol()

- 비트마스크 DP풀이
- 34596KB / 524ms

In [ ]:
PRICE = [[100, 70, 40, 0], [70, 50, 30, 0], [40, 30, 20, 0], [0, 0, 0, 0]]

import collections
INF = 1 << 30
class MCMF:
  def __init__(self, n) :
    self.len = n
    self.G = [[] for _ in range(n)]
    self.C = [[0] * n for _ in range(n)]
    self.F = [[0] * n for _ in range(n)]
    self.W = [[0] * n for _ in range(n)]

  def addEdge(self, u, v, c, w) :
    self.G[u].append(v)
    self.G[v].append(u)
    self.C[u][v] = c
    self.W[u][v] = w
    self.W[v][u] = -w

  def SPFA(self, src, snk):
    mc = mf = 0
    while True :
      vis = [False] * self.len
      dist = [INF] * self.len
      par = [-1] * self.len

      Q = collections.deque([src])
      dist[src] = 0
      vis[src] = True
      while Q :
        u = Q.popleft()
        vis[u] = False
        for v in self.G[u] :
          if self.C[u][v] - self.F[u][v] > 0 and dist[v] > dist[u] + self.W[u][v] :
            dist[v] = dist[u] + self.W[u][v]
            par[v] = u
            if not vis[v] :
              vis[v] = True
              Q.append(v)

      if par[snk] == -1 : break

      flow = INF
      u = snk
      while u != src :
        flow = min(flow, self.C[par[u]][u] - self.F[par[u]][u])
        u = par[u]
      
      u = snk
      while u != src :
        mc += flow * self.W[par[u]][u]
        self.F[par[u]][u] += flow
        self.F[u][par[u]] -= flow
        u = par[u]

      mf += 1

    return mc, mf

def sol() :
  N = int(input())
  G = [[*map(lambda x: ord(x) - 65 if x != 'F' else 3, input().rstrip())] for _ in range(N)]
  V = N * N + 2
  src, snk = V - 2, V - 1
  mcmf = MCMF(V)

  idx = lambda x, y: x * N + y
  for x, l in enumerate(G):
    for y, v in enumerate(l):
      if (x+y) % 2 : continue
      mcmf.addEdge(src, idx(x, y), 1, 0)
      mcmf.addEdge(idx(x, y), snk, 1, 0)
      for dx, dy in ((0, 1), (0, -1), (1, 0), (-1, 0)) :
        nx, ny = x + dx, y + dy
        if nx < 0 or nx >= N or ny < 0 or ny >= N : continue
        mcmf.addEdge(idx(x, y), idx(nx, ny), 1, -PRICE[v][G[nx][ny]])
        mcmf.addEdge(idx(nx, ny), snk, 1, 0)
  
  mc, mf = mcmf.SPFA(src, snk)
  print(-mc)
  
sol()

- MCMF풀이
- 34344KB, 96ms